In [1]:
import ee
ee.Initialize()
from iteru import *
from ipywidgets import *

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [60]:
col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

In [61]:
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'Africa'))

In [62]:
Map.add_ee_layer(mask)

In [63]:
region = ee.Geometry.Polygon(
  [[[-18.698368046353494, 38.1446395611524],
    [-18.698368046353494, -36.16300755581617],
    [52.229366328646506, -36.16300755581617],
    [52.229366328646506, 38.1446395611524]]],
  None, False
)

In [68]:
def get_doy(img):
    doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
    return img.set('doy', doy)
col = col.map(get_doy)

In [106]:
distinctDOY = col.filterDate('2013-01-01', '2014-01-01')

In [107]:
filter = ee.Filter.equals('leftField', 'doy', 'rightField','doy')

In [117]:
join = ee.Join.saveAll('doy_matches')

In [118]:
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter))

In [119]:
def reduce_com(img):
    doyCol = ee.ImageCollection.fromImages(img.get('doy_matches'))
    return doyCol.reduce(ee.Reducer.median())

In [120]:
comp = joinCol.map(reduce_com)

In [135]:
visParams = {
  'min': 0.0,
  'max': 9000.0,
  'bands': [
    'B4','B3','B2'
  ],
}

In [136]:
def vis(img):
    return img.visualize(visParams).clip(mask)

In [137]:
rgbVis = distinctDOY.map(vis)

In [138]:
gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:3857',
  'framesPerSecond': 10}

In [139]:
print(rgbVis .getVideoThumbURL(gifParams))

EEException: Error in map(ID=2013_01_01):
Image.visualize: Expected a string or list of strings for field 'bands'.

In [143]:
ui.Thumbnail(distinctDOY, gifParams)

NameError: name 'ui' is not defined